In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import os

In [2]:
def save_world_to_database(df_world):
    connection_string = "postgres:postgres@localhost:5432/corona_db"
    engine = create_engine(f'postgresql://{connection_string}')
    connection = engine.connect()
    connection.execute( '''TRUNCATE TABLE daily_stats_world''' )
    connection.close()
    df_world.to_sql(name='daily_stats_world', con=engine, if_exists='append', index=False)

In [3]:
# Start here to process the world data (only call save_world_to_database)
url_world = "static/world_data/df_world_all.csv"
df_world = pd.read_csv(url_world)
df_world.head()
save_world_to_database(df_world)

In [4]:
# query_str = open('static/sql/test_world.sql')
query_str = open('static/sql/world_top_10.sql')
query_text = ""
for text in query_str:
    query_text = query_text + text

connection_string = "postgres:postgres@localhost:5432/corona_db"
engine = create_engine(f'postgresql://{connection_string}')    
print(query_text)
df_query = pd.read_sql_query(query_text, con=engine)

df_query


select date, 
	country,
	sum(conf_count) confirmed, 
	sum(cured_count) cured,
	sum(dead_count) dead
from daily_stats_world 
group by date, country 
order by date


,date,country,confirmed,cured,dead
0,2020-01-22,France,0,0,0
1,2020-01-22,Brazil,0,0,0
2,2020-01-22,India,0,0,0
3,2020-01-22,Japan,2,0,0
4,2020-01-22,Russia,0,0,0
...,...,...,...,...,...
1759,2020-02-26,India,3,3,0
1760,2020-02-26,Iran,139,49,19
1761,2020-02-26,Russia,2,2,0
1762,2020-02-26,South Korea,1261,22,12


In [13]:
result_set = []
result_set = engine.execute("select  \
    country, \
    max(date), \
    sum(conf_count) confirmed, \
    sum(cured_count) cured, \
    sum(dead_count) dead, \
    (sum(conf_count) + sum(cured_count) + sum(dead_count)) total \
    from daily_stats_world \
    where date = (select max(date ) from daily_stats_world) \
    group by country \
    order by total desc \
    limit 10")

result_sum = []
result_sum = engine.execute("select  max(date), \
    sum(conf_count) confirmed, \
    sum(cured_count) cured, \
    sum(dead_count) dead, \
    (sum(conf_count) + sum(cured_count) + sum(dead_count)) total \
    from daily_stats_world  \
    where date = (select max(date ) from daily_stats_world)") 

pct_count = 0

for i in result_sum:
    print(i)
    pct_conf = round(i[1]/i[4]*100)
    pct_cured = round(i[2]/i[4]*100)
    pct_dead = round(i[3]/i[4]*100)


print(pct_conf)
print(pct_cured)
print(pct_dead)

result_sum.append(pct_conf)

('2020-02-26', 81397, 30384, 2770, 114551)
71
27
2


AttributeError: 'ResultProxy' object has no attribute 'append'